In [ ]:
import numpy
import pandas
import math

IrisData = pandas.read_csv('bezdekIris.csv', header=-1)
# IrisDict = {"Iris-setosa":0, "Iris-versicolor":1, "Iris-virginica":2} # Do we even need this? (Answered 'no', commented out)
# all columns except last one are floats

# Attributes:
#    1. sepal length in cm
#    2. sepal width  in cm
#    3. petal length in cm
#    4. petal width  in cm
#    5. class: 

# Summary Statistics: # Not completely relevant to our study, but still nice to have around.
#                  Min  Max   Mean    SD   Class Correlation
#    sepal length: 4.3  7.9   5.84  0.83    0.7826   
#    sepal width:  2.0  4.4   3.05  0.43   -0.4194
#    petal length: 1.0  6.9   3.76  1.76    0.9490  (high!)
#    petal width:  0.1  2.5   1.20  0.76    0.9565  (high!)

# Some high level plays; thanks to stackoverflow
# @ https://stackoverflow.com/questions/17071871/select-rows-from-a-dataframe-based-on-values-in-a-column-in-pandas
IrisData = IrisData.set_index([4])
SeD = IrisData.loc['Iris-setosa']
VeD = IrisData.loc['Iris-versicolor']
ViD = IrisData.loc['Iris-virginica']

del IrisData

MeanByClass = numpy.array([SeD.mean().as_matrix(), VeD.mean().as_matrix(), ViD.mean().as_matrix()])
StdByClass = numpy.array([SeD.std().as_matrix(), VeD.std().as_matrix(), ViD.std().as_matrix()])
StatsByClass = numpy.dstack((MeanByClass,StdByClass))
    # once again, thanks to stackoverflow
    # @ https://stackoverflow.com/questions/17960441/in-numpy-how-to-zip-two-2-d-arrays
    # here, indexing is as follows: [class, attribute, mean/std]
    # so [1,2,0] is "2nd class: Ve, 3rd att: pl, 1st opt: mean"
    
    # At this point, we can delete all data frames (SeD, VeD, ViD) and single stat by classes.
    # StatsByClass is, for all intents and purposes, all we need.

def NormalProb(point, mean=0., std=1., interval=0.01):
    # For details see:
    # http://mathworld.wolfram.com/NormalDistribution.html
    # http://mathworld.wolfram.com/Erf.html
    # https://docs.python.org/dev/library/math.html#math.erf
    xlower = ((point-mean)-interval*0.5)/std
    xupper = ((point-mean)+interval*0.5)/std
    return (math.erf(xupper/math.sqrt(2.))-math.erf(xlower/math.sqrt(2.)))/2.
    
def FlowerDetermination(sl=0,sw=0,pl=0,pw=0):
    """Accepts up to 4 floats, returns probability of each class being in its range.
    Since we have a continuous probability distribution function, but user is
    entering single values, results are taken in an interval of length 0.1
    around point x0, i.e. in x0+-0.05.
    
    Though the data set is finite, and has a maximum and minimum, we are making a
    normalized approximation, therefore any number is (theoritically) possible. However,
    since all variables correspond to physical measurements (lengths and widths) any
    non-positive entries will be omitted.
    
    Following is a listing of our parameters and their real-world correspondants:
        sl: Sepal Length
        sw: Sepal Width
        pl: Petal Length
        pw: Petal Width"""
    
    # Once again, we have 4 numbers (sl, sw, pl, pw) and we will return an array of 3 numbers:
    # p1 = p(1|(sl+-0.05, sw+-0.05, pl+-0.05, pw+-0.05))
    #    = p((sl+-0.05, sw+-0.05, pl+-0.05, pw+-0.05)|1)*p(1) # Actually in proportion.
    #    = p(sl+-0.05|1)*p(sw+-0.05|1)*p(pl+-0.05|1)*p(pw+-0.05|1)*p(1) # From 'Naive Assumption'.
    #     # Here, p(Class)=1/3 for all Class, so can be omitted.
    # To find, let's say, p(sl+-0.05|1), we will need to look at normal distribution
    # with mean StatsByClass[0,0,0]=M00 and std StatsByClass[0,0,1]=S00
    # In other words, area of std normal distribution b/w (sl-0.05-M00,sl+0.05-M00)/S00
    # For simplicity, let us calculate these values anyway, then multiply or not depending on parameter's inclusion
    
    param = [sl, sw, pl, pw]
    pS = numpy.zeros((4,3))
    for i in range(4):
        for k in range(3):
            pS[i,k] = NormalProb(point=param[i], mean=StatsByClass[k,i,0], std=StatsByClass[k,i,1])
    p = numpy.ones(3)
    for i in range(4):
        if param[i]>0:
            p = p*pS[i]
    return p/numpy.sum(p)